In [4]:
!mkdir dataset

In [2]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other




  0%|          | 0.00/80.9M [00:00<?, ?B/s]
  1%|          | 1.00M/80.9M [00:01<02:05, 667kB/s]
  2%|▏         | 2.00M/80.9M [00:01<01:05, 1.26MB/s]
  4%|▎         | 3.00M/80.9M [00:02<00:43, 1.87MB/s]
  5%|▍         | 4.00M/80.9M [00:02<00:32, 2.45MB/s]
  6%|▌         | 5.00M/80.9M [00:02<00:29, 2.72MB/s]
  7%|▋         | 6.00M/80.9M [00:03<00:27, 2.82MB/s]
  9%|▊         | 7.00M/80.9M [00:03<00:27, 2.79MB/s]
 10%|▉         | 8.00M/80.9M [00:03<00:27, 2.78MB/s]
 11%|█         | 9.00M/80.9M [00:04<00:28, 2.69MB/s]
 12%|█▏        | 10.0M/80.9M [00:04<00:27, 2.70MB/s]
 14%|█▎        | 11.0M/80.9M [00:04<00:26, 2.75MB/s]
 15%|█▍        | 12.0M/80.9M [00:05<00:25, 2.78MB/s]
 16%|█▌        | 13.0M/80.9M [00:05<00:25, 2.80MB/s]
 17%|█▋        | 14.0M/80.9M [00:06<00:25, 2.71MB/s]
 19%|█▊        | 15.0M/80.9M [00:06<00:28, 2.46MB/s]
 20%|█▉        | 16.0M/80.9M [00:06<00:25, 2.64MB/s]
 21%|██        | 17.0M/80.9M [00:07<00:24, 2.77MB/s]
 22%|██▏       | 18.0M/80.9M [00:07<00:22, 3.00MB/s]
 2

In [6]:
from zipfile import ZipFile

dataset = './sentiment140.zip'

with ZipFile(dataset, 'r') as zip_ref:
    zip_ref.extractall('./dataset/')
print('Extraction complete!')

Extraction complete!


In [7]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
columns = ['target', 'ids', 'date', 'flag', 'user', 'text']


In [43]:
# loading the data
data = pd.read_csv('./dataset/training.1600000.processed.noemoticon.csv', names=columns, encoding='ISO-8859-1')

In [38]:
data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [44]:
data.replace({'target': {4: 1}}, inplace=True)
data.target.value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [40]:
stemmer = PorterStemmer()
def stemming(tweet):
    tweet = re.sub('[^a-zA-Z]', ' ', tweet)
    tweet = tweet.lower()
    tweet = tweet.split()
    tweet = [stemmer.stem(word) for word in tweet if word not in set(stopwords.words('english'))]
    tweet = ' '.join(tweet)
    return tweet

In [45]:
#reducing the size of the dataset to 200000 equal number of positive and negative tweets
data_pos = data[data['target'] == 1].iloc[:100000]
data_neg = data[data['target'] == 0].iloc[:100000]
data = pd.concat([data_pos, data_neg], axis=0)

In [46]:
data['text'] = data['text'].apply(stemming)

In [48]:
print(data['text'])

800000                     love health uandpet u guy r best
800001        im meet one besti tonight cant wait girl talk
800002    darealsunisakim thank twitter add sunisa got m...
800003    sick realli cheap hurt much eat real food plu ...
800004                         lovesbrooklyn effect everyon
                                ...                        
99995           son develop new habit wake second pot coffe
99996                     look like router broke tweet fone
99997              realli dont want colleg right wish sunni
99998                                    flossa offer pepto
99999                          josiehobo would sooooo revis
Name: text, Length: 200000, dtype: object


In [52]:
x = data['text'].values
y = data['target'].values

print(x)
print(y)

['love health uandpet u guy r best'
 'im meet one besti tonight cant wait girl talk'
 'darealsunisakim thank twitter add sunisa got meet hin show dc area sweetheart'
 ... 'realli dont want colleg right wish sunni' 'flossa offer pepto'
 'josiehobo would sooooo revis']
[1 1 1 ... 0 0 0]


In [54]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2)

In [55]:
print(X_train.shape, X_test.shape)


(160000,) (40000,)


In [56]:
# converting text data to numerical data

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [57]:
# training the model
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [58]:
X_train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, X_train_pred)

In [59]:
print('Training accuracy:', train_accuracy)

Training accuracy: 0.8174125


In [60]:
X_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test,X_test_pred)

In [61]:
print('Test accuracy:', test_accuracy)

Test accuracy: 0.7692
